In [ ]:
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-research')

In [ ]:
import numpy as np

from qiskit import QuantumCircuit, transpile
from qiskit.utils import QuantumInstance
from qiskit.providers.aer.backends import AerSimulator
from qiskit.aqua import aqua_globals
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.ml.circuit.library import RawFeatureVector
sys.path.append('../../')
from qclib.machine_learning.baa_feature_vector import BaaFeatureVector
from qclib.machine_learning.vqc import VQC
from qclib.machine_learning.datasets import digits
from qclib.state_preparation.baa_schmidt import initialize as baa_schmidt

seed = 42

In [ ]:
# Dataset load

feature_dim = 64
sample_total, training_input, test_input, class_labels = digits.load(classes=[1,3],
                                                                     training_size=40,
                                                                     test_size=10,
                                                                     random_seed=seed)
for i in training_input:
    print(i, len(training_input[i]))
print()
for i in test_input:
    print(i, len(test_input[i]))

In [ ]:
# Two-local parameterized circuit model

n_qubits = int(np.ceil(np.log2(feature_dim)))
var_form = TwoLocal(n_qubits, ['ry', 'rz'], 'cz', reps=1)
var_form.decompose().draw(output='mpl')

In [ ]:
# VQC experiments

def ideal_instance():
    ideal = AerSimulator()
    return QuantumInstance(ideal, shots=1024, seed_simulator=seed, seed_transpiler=seed)

def noisy_instance():
    jakarta = provider.get_backend("ibmq_jakarta")
    noisy = AerSimulator.from_backend(jakarta)
    return QuantumInstance(noisy, shots=1024, seed_simulator=seed, seed_transpiler=seed)

aqua_globals.random_seed = seed

optimizer = SPSA(maxiter=200)

training_size = sum([ len(val) for key, val in training_input.items() ] )
batch_size = int(training_size * 0.1)
print(training_size)
print(batch_size)

#feature_map = RawFeatureVector(feature_dimension=feature_dim)
feature_map = BaaFeatureVector(feature_dimension=feature_dim, 
                               strategy='brute_force',
                               max_fidelity_loss=0.1,
                               use_low_rank=True)

vqc = VQC(optimizer, feature_map, var_form, training_input, test_input, None, minibatch_size=batch_size)

#quantum_instance = noisy_instance()
quantum_instance = ideal_instance()

acc = []
for i in range(1):
    print(i)

    result = vqc.run(quantum_instance)
    acc.append(result["testing_accuracy"])

    print(f'Test accuracy: {result["testing_accuracy"]}')

print('AVG:', sum(acc) / len(acc))
print('STD:', np.std(acc))

In [ ]:
# Average number of CNOTs, depth, and fidelity

def _fidelity(input_state, transpiled_circuit):
    backend = AerSimulator()
    transpiled_circuit.save_statevector()
    ket = backend.run(transpiled_circuit).result().get_statevector()
    bra = np.conj(input_state)

    return np.abs(bra.dot(ket))**2

def _counts(input_state, l, result, strategy='brute_force'):
    if strategy == 'qiskit':
        n_qubits = int(np.log2(len(input_state)))
        circuit = QuantumCircuit(n_qubits)
        circuit.initialize(input_state)
    else:
        circuit = baa_schmidt(input_state, use_low_rank=True, max_fidelity_loss=l, strategy=strategy)
    transpiled_circuit = transpile(circuit, basis_gates=['u1','u2','u3', 'cx'], optimization_level=3)
    
    count_ops = transpiled_circuit.count_ops()
    n_cx = 0
    if 'cx' in count_ops:
        n_cx = count_ops['cx']
    n_dp = transpiled_circuit.depth()

    fidelity = _fidelity(input_state, transpiled_circuit)

    result.append([l, n_cx, n_dp, fidelity])
        
def _grid_search(input_state, fidelity_loss=None):
    result = []
    if fidelity_loss is None:
        fidelity_loss = [i/10 for i in range(7)]

    n = int(np.log2(len(input_state)))
    for l in fidelity_loss:
        _counts(input_state, l=l, result=result, strategy='brute_force')
    
    return result

def _print_results(results):
    fidelity_loss = [r[0] for r in results[0]]

    n_cx = {}
    n_dp = {}
    fidelity = {}
    for l in fidelity_loss:
        n_cx[l] = []
        n_dp[l] = []
        fidelity[l] = []

    for result in results:
        for l in fidelity_loss:
            n_cx[l].extend([r[1] for r in result if r[0]==l])
            n_dp[l].extend([r[2] for r in result if r[0]==l])
            fidelity[l].extend([r[3] for r in result if r[0]==l])
    
    print('AVG:')
    for l in fidelity_loss:
        avg_cx = sum(n_cx[l]) / len(n_cx[l])
        avg_dp = sum(n_dp[l]) / len(n_dp[l])
        avg_fidelity = sum(fidelity[l]) / len(fidelity[l])
        print('l={3}\tCNOTs={0}\tdepth={1}\tfidelity={2}'.format(avg_cx, avg_dp, avg_fidelity, l))
    
    print('STD:')
    for l in fidelity_loss:
        std_cx = np.std(n_cx[l])
        std_dp = np.std(n_dp[l])
        std_fidelity = np.std(fidelity[l])
        print('l={3}\tCNOTs={0}\tdepth={1}\tfidelity={2}'.format(std_cx, std_dp, std_fidelity, l))

results = []
for i in training_input:
    print(i)
    for input_state in training_input[i]:
        results.append(_grid_search(input_state))
print()
for i in test_input:
    print(i)
    for input_state in training_input[i]:
        results.append(_grid_search(input_state))

_print_results(results)